In [ ]:
# !pip install -r requirements.txt

In [ ]:
# import os
# os.environ['TRANSFORMERS_CACHE'] = '/home/ec2-user/SageMaker/cache/'


# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("csdc-atl/baichuan-7B-chat", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("csdc-atl/baichuan-7B-chat", trust_remote_code=True).half().cuda()
# response, history = model.chat(tokenizer, "“面朝大海，春暖花开”的出处是？", history=[])
# print(response)
# response, history = model.chat(tokenizer, "能不能把这一首诗完整背诵一下", history=history)
# print(response)


# # from transformers import AutoModelForCausalLM, AutoTokenizer

# # tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/Baichuan-7B", trust_remote_code=True)
# # model = AutoModelForCausalLM.from_pretrained("baichuan-inc/Baichuan-7B", device_map="auto", trust_remote_code=True)
# # #,offload_folder
# # inputs = tokenizer('登鹳雀楼->王之涣\n夜雨寄北->', return_tensors='pt')
# # inputs = inputs.to('cuda:0')
# # pred = model.generate(**inputs, max_new_tokens=64,repetition_penalty=1.1)
# # print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

In [13]:
# import boto3

# client = boto3.client('sagemaker')

# response = client.delete_endpoint_config(EndpointConfigName='pytorch-inference-baichuan-v1')

In [3]:
!rm -rf src
!mkdir src

In [4]:
%%writefile ./src/serving.properties
engine=DeepSpeed
option.tensor_parallel_degree=1
option.s3url=s3://sagemaker-us-east-1-310850127430/baichuan7b/
batch_size=2
max_batch_delay=50

Writing ./src/serving.properties


In [4]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)
print(region_name)


if "cn-" in region_name:
    with open('./code/requirements.txt', 'r') as original: data = original.read()
    with open('./code/requirements.txt', 'w') as modified: modified.write("-i https://pypi.tuna.tsinghua.edu.cn/simple\n" + data)

!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'llm_chinese')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'llm_chinese')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

model_name = None
entry_point = 'inference.py'
# framework_version = '1.13.1'
# py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'420', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}



url = f'763104351884.dkr.ecr.{region_name}.amazonaws.com/huggingface-pytorch-inference:2.0.0-transformers4.28.1-gpu-py310-cu118-ubuntu20.04'
from sagemaker.huggingface.model import HuggingFaceModel
model = HuggingFaceModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    # framework_version = framework_version, 
    # py_version = py_version,
    # env = model_environment
    image_uri=url
)

endpoint_name = 'hf-inference-baichuan-v1'
instance_type='ml.g5.4xlarge' 

instance_count = 1


import boto3

client = boto3.client('sagemaker')
try:
    response = client.delete_endpoint_config(EndpointConfigName=endpoint_name)
except:
    pass


from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

arn:aws:iam::310850127430:role/NotebookStack-SmartSearchNotebookRole6F6BB12B-XTY1WIG30A1B
sagemaker-us-east-2-310850127430
us-east-2
dummy
upload: ./model.tar.gz to s3://sagemaker-us-east-2-310850127430/llm_chinese/assets/model.tar.gz
----------!

In [6]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = sagemaker.predictor.Predictor(endpoint_name)
predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

inputs= {
    "ask": "晚上睡不着应该怎么办"

}

response = predictor.predict(inputs)
print(response["answer"])

晚上睡不着可能是因为身体疲劳、压力过大、焦虑、饮食不当等原因。以下是一些帮助您入睡的建议：

1. 放松身体：尝试进行深呼吸、冥想或瑜伽等放松活动，以帮助您放松身体和思维。

2. 创造一个舒适的睡眠环境：确保您的卧室安静、黑暗、凉爽和舒适。

3. 避免刺激性物质：避免饮用咖啡因、酒精和尼古丁等刺激性物质，因为它们会影响您的睡眠。

4. 建立规律的睡眠时间：尽量在同一时间入睡和起床，以帮助您的身体建立规律的睡眠模式。

5. 避免午睡：如果您需要午睡，请确保在下午3点之前完成，以避免晚上难以入睡。

6. 避免使用电子设备：在睡觉前避免使用电子设备，因为它们会刺激您的思维和眼睛。

7. 寻求帮助：如果您尝试了以上建议但仍然无法入睡，请考虑咨询医生或专业人士的建议。
